# VTT AI Translator
## About
This is a notebook to translate VTT subtitle / caption files fluently using the OpenAI API and webvtt. It can be switched to Deepseek easily in the future for more cost-efficient processing. Made by Connor Wright for Georgia Tech's Buzz Studios Filmmaking Club. 

## How to Use 
* Clone the repo
* Change the folder / file paths to the respective vtt
* Set a language using the ISO language code
* Put in an OpenAI API key (or ask for mine)
* Run all the cells

In [ ]:
### pip installations
%pip install openai
%pip install webvtt-py

In [38]:
### Package imports
from openai import OpenAI
import os
import webvtt

In [69]:
### Setup variables
## Set file paths / language output 
folder_path = "/Users/connorwright/Downloads/GT.CS.CodeFiles/BuzzStudios/Assets/Subtitles/"
vtt_name = "frisbee-fables-cc.vtt"
trans_lang = "es"

## Set API Key 
api_key = ""

vtt_path = os.path.join(folder_path, vtt_name)

In [ ]:
### Turn original captions into single string for GPT input 
captions = []
for caption in webvtt.read(vtt_path):
    #print(caption.start)  # start timestamp in text format
    #print(caption.end)  # end timestamp in text format
    #print(caption.text)  # caption text
    #print(caption.voice)  # voice span if present
    captions.append(caption.text)

captions = "\n".join(captions) if isinstance(captions, list) else str(captions)
print(captions)


In [ ]:
### Setup OpenAI client and context
#client = OpenAI(api_key="", base_url="https://api.deepseek.com")
client = OpenAI(api_key=api_key)

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a professional subtitle translator. \
            You will only receive a string transcription of a vtt file containing subtitles in English. \
            You will only output a translation of the subtitles and bracketed actions. \
            Do not add anything else to your reply.\
            Do not merge sentences, translate each line individually. \
            Return the translated subtitles in the same order and length as the input. \
            Your steps are as follows: \
            1. Parse the input subtitles \
            2. Translate the input subtitles into {trans_lang} \
            3. Alter the translated subtitles into more fluent sentences \
            4. Use the setResult method to output the translated subtitles as a string[] \
         "},
        {"role": "user", "content": captions}
    ]
)

print(response)

In [ ]:
### Save translated captions as new vtt file 

## Get GPT response as string, split into list
trans_str = str(response.choices[0].message.content)
trans_list = trans_str.split("\n")
print(trans_list)

## Edit caption files to match translations, accounting for multi-line texts 
trans_vtt = webvtt.read(vtt_path)
line_index = 0
for caption in trans_vtt:
    num_lines = len(caption.text.split("\n"))
    trans_lines = trans_list[line_index:line_index+num_lines]
    caption.text = "\n".join(trans_lines)
    line_index += num_lines

## Save as new file w/ specified language name 
trans_filename = str(os.path.splitext(vtt_name)[0]) + '-' + str(trans_lang) + '.vtt'
trans_path = os.path.join(folder_path, trans_filename)
trans_vtt.save(trans_path)